In [1]:
from modsim import *

pint.unit.build_unit_class.<locals>.Unit

In [165]:
class Vector(np.ndarray):
    """Represents a vector, usually in 2D or 3D.
    
    Based on https://gist.github.com/eigencoder/c029d7557e1f0828aec5
    """
    def __new__(cls, *args):
        """
        cls:  class object
        args: tuple of coordinates
        """
        if len(args) < 2:
            raise ValueError('Vector must contain at least two elements.')
        
        # if any of the elements have units, store the first one
        for elt in args:
            units = getattr(elt, 'units', None)
            if units:
                break
            
        if units:
            # if there are units, remove them
            args = [getattr(elt, 'magnitude', elt) for elt in args]
            
        obj = np.asarray(args).view(cls)    
        return obj
    
    def __mul__(self, other):
        if isinstance(other, UNITS.Unit):
            return VectorQuantity(self, other)
        else:
            return super().__mul__(other)
            
    def __div__(self, other):
        if isinstance(other, UNITS.Unit):
            return VectorQuantity(self, 1/other)
        else:
            return super().__truediv__(other)
        
    __truediv__ = __div__
            
    @property
    def x(self):
        return self[0]

    @property
    def y(self):
        return self[1]

    @property
    def z(self):
        return self[2]
    
    def __repr__(self):
        t = map(str, self)
        s = ', '.join(t)
        return '%s(%s)' % (self.__class__.__name__, s)
        
    def __eq__(self, other):
        return np.array_equal(self, other)

    def __ne__(self, other):
        return not np.array_equal(self, other)

    def __iter__(self):
        for x in np.nditer(self):
            yield x.item()

    @property
    def mag(self):
        """
        """
        return np.sqrt(np.dot(self, self))

    @property
    def mag2(self):
        """
        """
        return np.dot(self, self)

    def hat(self):
        """
        """
        return self / self.mag

    @property
    def angle(self):
        """
        """
        return np.arctan2(self.y, self.x)

    def polar(self):
        return self.mag, self.angle

    def dot(self, other):
        """
        """
        return np.dot(self, other)

    def cross(self, other):
        """
        """
        return np.cross(self, other)

    def proj(self, other):
        """
        """
        print('hello1')
        b_hat = other.hat()
        return self.dot(b_hat) * b_hat

    def comp(self, other):
        """
        """
        return self.dot(other.hat())

    def dist(self, other):
        """Euclidean distance from self to other.
        """
        return np.linalg.norm(self - other)

    def diff_angle(self, other):
        """
        """
        #TODO: see http://www.euclideanspace.com/maths/algebra/vectors/angleBetween/
        raise NotImplementedError()

In [192]:
def __mul__(self, other):
        if isinstance(other, UNITS.Unit):
            return VectorQuantity(self, self.units * other)
        
        if isinstance(other, UNITS.Quantity):
            prod = super().__mul__(other.magnitude)
            prod._units = self.units * other.units
            return prod
        else:
            prod = super().__mul__(other)
            return prod
        
    def __div__(self, other):
        if isinstance(other, UNITS.Unit):
            return VectorQuantity(self, self.units / other)
        
        if isinstance(other, UNITS.Quantity):
            quot = super().__div__(other.magnitude)
            quot._units = self.units / other.units
            return quot
        else:
            quot = super().__div__(other)
            return quot
        
    __truediv__ = __div__

IndentationError: unindent does not match any outer indentation level (<ipython-input-192-40c5da61710c>, line 13)

In [193]:
class VectorQuantity(Vector, UNITS.Quantity):
    
    def __new__(cls, vector, units):
        """
        cls:  class object
        args: tuple of coordinates
        """
        obj = np.asarray(vector).view(cls)
        obj._units = units    
        return obj
    
    def __repr__(self):
        return super().__repr__() + ' ' + str(self.units)
    
    def __mul__(self, other):
        if isinstance(other, UNITS.Unit):
            return VectorQuantity(self, self.units * other)
        
        if isinstance(other, UNITS.Quantity):
            prod = super().__mul__(other.magnitude)
            prod._units = self.units * other.units
            return prod
        else:
            prod = super().__mul__(other)
            return prod
        
    def __div__(self, other):
        if isinstance(other, UNITS.Unit):
            return VectorQuantity(self, self.units / other)
        
        if isinstance(other, UNITS.Quantity):
            quot = super().__div__(other.magnitude)
            quot._units = self.units / other.units
            return quot
        else:
            quot = super().__div__(other)
            return quot
        
    __truediv__ = __div__
            
    #def __getitem__(self, key):
    #    return super().__getitem__(key) * self.units

    @property
    def x(self):
        return self[0] * self.units

    @property
    def y(self):
        return self[1] * self.units

    @property
    def z(self):
        return self[2] * self.units
    
    @property
    def mag(self):
        return super().mag * self.units
    
    @property
    def mag2(self):
        return super().mag2 * self.units

    def hat(self):
        return super().hat() * self.units

    def dot(self, other):
        prod = super().dot(other) * self.units
        try:
            return prod * other.units
        except AttributeError:
            return prod
        
    def cross(self, other):
        prod = super().cross(other) * self.units
        try:
            return prod * other.units
        except AttributeError:
            return prod
        
    def proj(self, other):
        """
        """
        print('hello1')
        return super().proj(other) / self.units
    
    def comp(self, other):
        """
        """
        return super().comp(other) / self.units

In [194]:
m = UNITS.m
N = UNITS.newton

In [195]:
a = Vector(3, 4)
b = Vector(1, 2)
a.cross(b)

array(2)

In [196]:
a = Vector(3, 4) * m
type(a)

__main__.VectorQuantity

In [197]:
a

VectorQuantity(3, 4) meter

In [198]:
a * m

VectorQuantity(3, 4) meter

In [199]:
a / m

TypeError: units must be of type str, Unit or UnitsContainer; not <class 'pint.unit.build_quantity_class.<locals>.Quantity'>.

In [173]:
a[0]

3

In [174]:
a.x

<Quantity(3, 'meter')>

In [175]:
a.y

<Quantity(4, 'meter')>

In [176]:
a.mag

<Quantity(5.0, 'meter')>

In [177]:
a.mag2

<Quantity(25, 'meter')>

In [178]:
type(a)

__main__.VectorQuantity

In [179]:
type(a.mag)

pint.unit.build_quantity_class.<locals>.Quantity

In [180]:
c = a * a.mag
c

VectorQuantity(15.0, 20.0) meter ** 2

In [181]:
c = a / a.mag
c

VectorQuantity(0.6, 0.8) dimensionless

In [182]:
c.x

<Quantity(0.6, 'dimensionless')>

In [183]:
ah = a.hat()
ah

VectorQuantity(0.6, 0.8) meter

In [184]:
print(ah)

[ 0.6  0.8]


In [185]:
ah.units

<Unit('meter')>

In [186]:
b = Vector(1, 2)

In [187]:
a.dot(b)

<Quantity(11, 'meter')>

In [188]:
b = Vector(1, 2) * N

In [189]:
a.dot(b)

<Quantity(11.0, 'meter * newton')>

In [190]:
c = a.cross(b)
c

<Quantity(2.0, 'meter * newton')>

In [191]:
c = a.proj(b)

hello1
hello1
maximum recursion depth exceeded while calling a Python object


In [133]:
c

<Quantity([ 2.2  4.4], '1 / meter')>

In [134]:
a.comp(b)

<Quantity(4.919349550499537, 'newton')>

In [135]:
x_hat = Vector(1, 0)
y_hat = Vector(0, 1)

In [136]:
a.comp(x_hat)

<Quantity(3.0, 'dimensionless')>

In [137]:
a.comp(y_hat)

<Quantity(4.0, 'dimensionless')>

In [138]:
a.angle

<Quantity(0.9272952180016122, 'radian')>

In [139]:
b.angle

<Quantity(1.1071487177940904, 'radian')>

In [140]:
Vector(1, 1).angle / pi

0.25

In [141]:
Vector(-1, 1).angle / pi

0.75

In [142]:
Vector(-1, -1).angle / pi

-0.75

In [143]:
Vector(1, -1).angle / pi

-0.25

In [144]:
Vector(1, -1).polar()

(1.4142135623730951, -0.78539816339744828)

In [145]:
x = [5, 3.5355, 0, -10]
x

[5, 3.5355, 0, -10]

In [146]:
y = [0, 3.5355, 10, 0]
y

[0, 3.5355, 10, 0]

In [147]:
theta, rho = cart2pol(x,y)

In [148]:
theta

array([ 0.        ,  0.78539816,  1.57079633,  3.14159265])

In [149]:
rho

array([  5.        ,   4.99995205,  10.        ,  10.        ])

In [150]:
theta = [0, pi/4, pi/2, pi]

In [151]:
rho = [5, 5, 10, 10]

In [152]:
x2, y2 = pol2cart(theta,rho)

In [153]:
print(x2)

[  5.00000000e+00   3.53553391e+00   6.12323400e-16  -1.00000000e+01]


In [154]:
print(y2)

[  0.00000000e+00   3.53553391e+00   1.00000000e+01   1.22464680e-15]


In [155]:
degree = UNITS.degree
kg = UNITS.kg
m = UNITS.m
s = UNITS.s

In [156]:
condition = Condition(x = 0 * m, 
                      y = 0 * m,
                      g = 9.8 * m/s**2,
                      mass = 145e-3 * kg,
                      diameter = 73e-3 * m,
                      rho = 1.2 * kg/m**3,
                      C_d = 0.3,
                      angle = 45 * degree,
                      velocity = 40 * m / s,
                      duration = 5 * s)

In [229]:
def make_system(condition):
    unpack(condition)
    
    theta = np.deg2rad(angle)
    vx, vy = pol2cart(theta, velocity)
    init = State(x=x, y=y, vx=vx, vy=vy)
    v = State(vx=vx, vy=vy)
    
    area = np.pi * (diameter/2)**2
    ts = linspace(0, duration, 101)
    
    return System(init=init, g=g, mass=mass, v=v,
                  area=area, rho=rho, C_d=C_d, ts=ts)

In [230]:
system = make_system(condition)
system.init

,value
x,0 meter
y,0 meter
vx,28.284271247461902 meter / second
vy,28.2842712474619 meter / second


In [235]:
v = system.v.values
v

array([<Quantity(28.284271247461902, 'meter / second')>,
       <Quantity(28.2842712474619, 'meter / second')>], dtype=object)

In [233]:
np.dot(v, v)

<Quantity(1600.0, 'meter ** 2 / second ** 2')>

In [222]:
def slope_func(state, t, system):
    x, y, vx, vy = state
    unpack(system)
    
    a_grav = np.array([0, -9.8]) * m / s**2

    v = np.array([vx.magnitude, vy.magnitude]) * m / s
    
    f_drag = -rho * np.linalg.norm(v) * v.units * v * C_d * area / 2
    a_drag = f_drag / mass

    a = a_grav + a_drag
    
    return v, a

In [223]:
slope_func(system.init, 0, system)

(<Quantity([ 28.28427125  28.28427125], 'meter / second')>,
 <Quantity([ -5.87820989 -15.67820989], 'meter / second ** 2')>)

In [161]:
a = 5 * m

In [162]:
b = Vector(1, 2) * s

In [163]:
b * a

VectorQuantity(5, 10) meter * second

In [164]:
a * b

maximum recursion depth exceeded in comparison


array([ 5, 10])